# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()
experiment_name = 'internet-firewall-hyper'
project_folder = './pipeline-project-hyper'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
internet-firewall-hyper,quick-starts-ws-138160,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cc-sta-ds12-hp"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

**Dataset**

**Internet Firewall Data Data Set**

**Data Set Information:**
There are 12 features in total. Action feature is used as a class. There are 4 classes in total. These are allow, action, drop and reset-both classes.


**Attribute Information:**
Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received

Link: https://archive.ics.uci.edu/ml/datasets/Internet+Firewall+Data

In [4]:
# NOTE: update the key to match the dataset name
found = False
key = "internet-firewall-dataset"
description_text = "Internet Firewall Data Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00542/log2.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C" : uniform(0.1,1),
                            "--max_iter": choice(50,100,150,200)})

#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=5,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=early_termination_policy,
    estimator=est
)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
#TODO: Submit your experiment

from azureml.core.experiment import Experiment


# experiment=Experiment(ws,experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_b0d0bb10-c0d3-4d2e-8e58-13a6591e4242
Web View: https://ml.azure.com/experiments/internet-firewall-hyper/runs/HD_b0d0bb10-c0d3-4d2e-8e58-13a6591e4242?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-138160/workspaces/quick-starts-ws-138160

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-08T16:25:37.535891][API][INFO]Experiment created<END>\n""<START>[2021-02-08T16:25:38.406864][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n"<START>[2021-02-08T16:25:38.8403659Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-08T16:25:38.718101][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [17]:
import joblib
import os
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])


Best Run Id:  HD_b0d0bb10-c0d3-4d2e-8e58-13a6591e4242_55

 Accuracy: 0.9825534079348932

 Regularization Strength:  0.3781905471361938

 Max iterations:  200


In [20]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
internet-firewall-hyper,HD_b0d0bb10-c0d3-4d2e-8e58-13a6591e4242_55,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_0976fbccfc46f487e9fc1a2da7dd9aa872d3f6b81e613844571ac9c6ea4ee97a_d.txt',
 'azureml-logs/65_job_prep-tvmps_0976fbccfc46f487e9fc1a2da7dd9aa872d3f6b81e613844571ac9c6ea4ee97a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_0976fbccfc46f487e9fc1a2da7dd9aa872d3f6b81e613844571ac9c6ea4ee97a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/107_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [19]:
joblib.dump(parameter_values, filename='best-hyperdrive.pkl')

['best-hyperdrive.pkl']

In [ ]:
#TODO: Save the best model
# joblib.dump(best_run, "./outputs/HPD_fitted_model.pkl")
from azureml.core import  Model

model_name = 'Internet-firewall-HPD-model'

model = Model.register(workspace=ws,model_name = model_name, model_path = 'best-hyperdrive.pkl')
print(model.name, model.id, model.version, sep='\t')

In [ ]:
compute_target.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service